# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
F. Walter  ->  F. Walter  |  ['F. Walter']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
H. Beuther  ->  H. Beuther  |  ['H. Beuther']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 84 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.11952


extracting tarball to tmp_2411.11952... done.


S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
E. Bañados  ->  E. Bañados  |  ['E. Bañados']
Y. Khusanova  ->  Y. Khusanova  |  ['Y. Khusanova']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 168 bibliographic references in tmp_2411.11952/aa_RLQSO_ALMA.bbl.
Issues with the citations
syntax error in line 977: '=' expected
Retrieving document from  https://arxiv.org/e-print/2411.12247
extracting tarball to tmp_2411.12247... done.
Retrieving document from  https://arxiv.org/e-print/2411.12257
extracting tarball to tmp_2411.12257...

...

extracting tarball to tmp_2411.12693...

 done.
  1: tmp_2411.12693/natbib.tex, 96 lines
  2: tmp_2411.12693/Orion_dust.tex, 504 lines


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2411.12693/Orion_dust.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'results_table' from 'tmp_2411.12693/results_table.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:41

T. Henning  ->  T. Henning  |  ['T. Henning']


Found 92 bibliographic references in tmp_2411.12693/Orion_dust.bbl.
Issues with the citations
syntax error in line 387: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.11952-b31b1b.svg)](https://arxiv.org/abs/2411.11952) | **The host galaxies of radio-loud quasars at z>5 with ALMA**  |
|| C. Mazzucchelli, et al. -- incl., <mark>S. Belladitta</mark>, <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2024-11-20*|
|*Comments*| *20 pages; 11 figures; accepted for publication in A&A*|
|**Abstract**|            The interaction between radio-jets and quasar host galaxies plays a paramount role in quasar/galaxy co-evolution. However, very little has been known so far about this interaction at very high-z. Here, we present new Atacama Large Millimeter/submillimeter Array (ALMA) observations in Band 7 and Band 3 of six radio-loud quasars' host galaxies at $z > 5$. We recover [CII] 158 $\mu$m line and underlying dust continuum emission at $>2\sigma$ for five sources, while we obtain upper limits for the CO(6-5) emission line and continuum for the remaining source. At the spatial resolution of our observations ($\sim$1.0"-1.4"), we do not recover perturbed/extended morphologies or kinematics, signatures of potential mergers. These galaxies already host large quantities of gas, with [CII]-based star formation rates of $30-400 M_{\odot} $yr$^{-1}$. Building their radio/sub-mm spectral energy distributions (SEDs), we find that in at least four cases the 1mm continuum intensity arises from a combination of synchrotron and dust emission, with an initial estimation of synchrotron contribution at 300 GHz of $\gtrsim$10%. We compare the properties of the sources inspected here with a large collection of radio-quiet sources from the literature, as well as a sample of radio-loud quasars from previous studies, at comparable redshift. We recover a potential mild decrease in $L_{\rm [CII]}$ for the radio-loud sources, which might be due to a suppression of the cool gas emission due to the radio-jets. We do not find any [CII]-emitting companion galaxy candidate around the five radio-loud quasars observed in Band 7: given the depth of our dataset, this result is still consistent with that observed around radio-quiet quasars. Further higher-spatial resolution observations, over a larger frequency range, of high-z radio-loud quasars hosts will allow for a better understanding of the physics of such sources.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.12257-b31b1b.svg)](https://arxiv.org/abs/2411.12257) | **Kinetic tomography of the Galactic plane within 1.25 kiloparsecs from the Sun. The interstellar flows revealed by HI and CO line emission and 3D dust**  |
|| J. D. Soler, et al. -- incl., <mark>H. Beuther</mark> |
|*Appeared on*| *2024-11-20*|
|*Comments*| *32 pages, 43 figures, submitted to Astronomy & Astrophysics (15NOV2024)*|
|**Abstract**|            We present a reconstruction of the line-of-sight motions of the local interstellar medium (ISM) based on the combination of a state-of-the-art model of the three-dimensional dust density distribution within 1.25 kpc from the Sun and the HI and CO line emission within Galactic latitudes $|b| < 5^{\circ}$. We use the histogram of oriented gradient (HOG) method to match the plane-of-the-sky 3D dust distribution across distances with the atomic and molecular line emission. We identify a significant correlation between the 3D dust model and the line emission. We employ this correlation to assign line-of-sight velocities to the dust across density channels and produce a face-on map of the local ISM radial motions with respect to the local standard of rest (LSR). We find that most of the material in the 3D dust model follows the large-scale pattern of Galactic rotation; however, we also report local departures from the rotation pattern with standard deviations of 12.1 and 6.1 km/s for the HI and CO line emission, respectively. The mean kinetic energy densities of these streaming motions are around 0.68 and 0.18 eV/cm$^{3}$ from either gas tracer. Assuming homogeneity and isotropy in the velocity field, these values are within a factor of a few of the total kinetic energy density. These kinetic energy values are roughly comparable to other energy densities, thus confirming the near-equipartition introduced by the feedback loops connecting the physical processes in the local ISM. Yet, we find energy and momentum overdensities of around a factor of ten concentrated in the Radcliffe Wave, the Split, and other local density structures. Although we do not find evidence of the Local Spiral Arm impact in these overdensities, their distribution suggests the influence of other large-scale effects that, in addition to supernova feedback, shape the energy distribution in the Solar neighborhood.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.12629-b31b1b.svg)](https://arxiv.org/abs/2411.12629) | **Estimating Dark Matter Halo Masses in Simulated Galaxy Clusters with Graph Neural Networks**  |
|| N. Garuda, J. F. Wu, D. Nelson, <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-11-20*|
|*Comments*| *9 pages, 4 figures, accepted at the NeurIPS ML4PS 2024 workshop*|
|**Abstract**|            Galaxies grow and evolve in dark matter halos. Because dark matter is not visible, galaxies' halo masses ($\rm{M}_{\rm{halo}}$) must be inferred indirectly. We present a graph neural network (GNN) model for predicting $\rm{M}_{\rm{halo}}$ from stellar mass ($\rm{M}_{*}$) in simulated galaxy clusters using data from the IllustrisTNG simulation suite. Unlike traditional machine learning models like random forests, our GNN captures the information-rich substructure of galaxy clusters by using spatial and kinematic relationships between galaxy neighbour. A GNN model trained on the TNG-Cluster dataset and independently tested on the TNG300 simulation achieves superior predictive performance compared to other baseline models we tested. Future work will extend this approach to different simulations and real observational datasets to further validate the GNN model's ability to generalise.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.12693-b31b1b.svg)](https://arxiv.org/abs/2411.12693) | **Peculiar Dust Emission within the Orion Molecular Cloud**  |
|| P. Nozari, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2024-11-20*|
|*Comments*| *Accepted to ApJ, 18 pages, 4 figures, 6 Tables*|
|**Abstract**|            It is widely assumed that dust opacities in molecular clouds follow a power-law profile with an index, $\beta$. Recent studies of the Orion Molecular Cloud (OMC) 2/3 complex, however, show a flattening in the spectral energy distribution (SED) at $ \lambda > 2$ mm implying non-constant indices on scales $\gtrsim$ 0.08 pc. The origin of this flattening is not yet known but it may be due to the intrinsic properties of the dust grains or contamination from other sources of emission. We investigate the SED slopes in OMC 2/3 further using observations of six protostellar cores with NOEMA from 2.9 mm to 3.6 mm and ALMA-ACA in Band 4 (1.9 -- 2.1 mm) and Band 5 (1.6 -- 1.8 mm) on core and envelope scales of $\sim 0.02 - 0.08$ pc. We confirm flattened opacity indices between 2.9 mm and 3.6 mm for the six cores with $\beta \approx -0.16 - 1.45$, which are notably lower than the $\beta$ values of $> 1.3$ measured for these sources on $0.08$ pc scales from single-dish data. Four sources have consistent SED slopes between the ALMA data and the NOEMA data. We propose that these sources may have a significant fraction of emission coming from large dust grains in embedded disks, which biases the emission more at longer wavelengths. Two sources, however, had inconsistent slopes between the ALMA and NOEMA data, indicating different origins of emission. These results highlight how care is needed when combining multi-scale observations or extrapolating single-band observations to other wavelengths.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.12247-b31b1b.svg)](https://arxiv.org/abs/2411.12247) | **Mass evolution of broad line regions to explain the luminosity variability of broad H$\alpha$ in the TDE ASASSN-14li**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2024-11-20*|
|*Comments*| *5 pages, 3 figures, accepted to be published in A&A Letters*|
|**Abstract**|            In this manuscript, an oversimplified model is proposed for the first time to explain the different variability trends in the observed broad H$\alpha$ emission line luminosity $L_{H\alpha}(t)$ and in the TDE model determined bolometric luminosity $L_{bol}(t)$ in the known TDE ASASSN-14li. Considering broad emission line regions (BLRs) lying into central accretion disk related to accreted materials onto central black hole in TDEs, mass evolution $M_{BLRs}(t)$ of central BLRs can be determined by the maximum mass $M_{BLRs,0}$ of central BLRs minus the corresponding accreted mass in a TDE. Meanwhile, through the simple linear dependence of Broad Balmer emission line luminosity on mass of BLRs, the mass evolution $M_{BLRs}(t)$ of central BLRs can be applied to describe the observed $L_{H\alpha}(t)$. Although the proposed model is oversimplified with only one free model parameter $M_{BLRs,0}$, the model with $M_{BLRs,0}\sim0.02{\rm M_\odot}$ can be applied to well describe the observed $L_{H\alpha}(t)$ in the TDE ASASSN-14li. Meanwhile, the oversimplified model can also be applied to roughly describe the observed $L_{H\alpha}(t)$ in the TDE ASASSN-14ae. The reasonable descriptions to the observed $L_{H\alpha}(t)$ in ASASSN-14li and ASASSN-14ae indicate the oversimplified model with only one free model parameter is probably efficient enough to describe mass evolutions of $M_{BLRs}$ related to central accreted debris in TDEs.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.11952/./fig_all_qsos_sed_202408.png', 'tmp_2411.11952/./fig_quant_distr_202408_cii-1.png', 'tmp_2411.11952/./fig_lfir_lciifir_20240826_v4.png']
copying  tmp_2411.11952/./fig_all_qsos_sed_202408.png to _build/html/
copying  tmp_2411.11952/./fig_quant_distr_202408_cii-1.png to _build/html/
copying  tmp_2411.11952/./fig_lfir_lciifir_20240826_v4.png to _build/html/
exported in  _build/html/2411.11952.md
    + _build/html/tmp_2411.11952/./fig_all_qsos_sed_202408.png
    + _build/html/tmp_2411.11952/./fig_quant_distr_202408_cii-1.png
    + _build/html/tmp_2411.11952/./fig_lfir_lciifir_20240826_v4.png
found figures ['', '', '', '', '', '']
file not found 
file not found 
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2411.12257.md
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
found figures ['tmp_2411.12629/./figs/distance_from_center_RMSE.png', 'tmp_2411.12629

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lya}{\mathrm{Ly}\ensuremath{\alpha}}$
$\newcommand{\civ}{C IV}$
$\newcommand{\mgii}{Mg II}$
$\newcommand{\cii}{[C II]}$
$\newcommand{\co}{CO(6-5)}$
$\newcommand{\cp}{C^{+}}$</div>



<div id="title">

# The host galaxies of radio--loud quasars at $z>$5 with ALMA

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.11952-b31b1b.svg)](https://arxiv.org/abs/2411.11952)<mark>Appeared on: 2024-11-20</mark> -  _20 pages; 11 figures; accepted for publication in A&A_

</div>
<div id="authors">

C. Mazzucchelli, et al. -- incl., <mark>S. Belladitta</mark>, <mark>E. Bañados</mark>, <mark>Y. Khusanova</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** The interaction between radio-jets and quasar host galaxies plays a paramount role in quasar/galaxy co-evolution. However, very little has been known so far about this interaction at very high $-z$ .  Here, we present new Atacama Large Millimeter/submillimeter Array (ALMA) observations in Band 7 and Band 3 of six radio-loud quasars' host galaxies at $z > 5$ . We recover $\cii$ 158 $\mu$ m line and underlying dust continuum emission at $>2\sigma$ for five sources, while we obtain upper limits for the $\co$ emission line and continuum for the remaining source.  At the spatial resolution of our observations ( $\sim$ 1 $\farcs$ 0-1 $\farcs$ 4), we do not recover perturbed/extended morphologies or kinematics, signatures of potential mergers.  These galaxies already host large quantities of gas ( $\sim10^{10}$ M $_{\odot}$ ), with $\cii$ luminosities of $L_{\rm \cii}\sim10^{8-9} L_{\odot}$ and $\cii$ -based star formation rates of $30-400 M_{\odot} $ yr $^{-1}$ .  Building their radio/sub-mm spectral energy distributions (SEDs), we find that in at least four cases the 1mm continuum intensity arises from a combination of synchrotron and dust emission, with an initial estimation of synchrotron contribution at 300 GHz of $\gtrsim$ 10 \% .  Assuming the case in which the continuum emission is due to only cold dust as upper limit, we obtain infrared (IR) luminosities of $L_{\rm IR}\sim10^{11-12} L_{\odot}$ .  We compare the properties of the sources inspected here with a large collection of radio-quiet sources from the literature, as well as a sample of radio-loud quasars from previous studies, at comparable redshift.  We recover a potential mild decrease in $L_{\rm \cii}$ for the radio-loud sources, which might be due to a suppression of the cool gas emission due to the radio-jets.  We do not find any $\cii$ emitting companion galaxy candidate around the five radio-loud quasars observed in Band 7: given the depth of our dataset, this result is still consistent with that observed around radio-quiet quasars.  Further higher-spatial resolution observations, over a larger frequency range, of high $-z$ radio-loud quasars hosts will allow for a better understanding of the physics of such sources.

</div>

<div id="div_fig1">

<img src="tmp_2411.11952/./fig_all_qsos_sed_202408.png" alt="Fig10" width="100%"/>

**Figure 10. -** Radio and (sub-)mm Spectral Energy Distributions (SED) of RL quasars reported in this work. We show the continuum emission at 1mm from our ALMA Band 7 observations (_ orange circles_), and the relative best fit modified black body dust emission (_ black line_). In the case of J2228+0110, we could retrieve only upper-limits from our ALMA Band 3 data, and we show the corresponding limits on its dust emission with a _ shaded grey area_. All modified black-body are calculated with T=47 K and $\beta=$1.6 (see Section \ref{sec:analysis_lum}). For each quasar, we report the radio observations obtained from a collection of literature data (see Section \ref{sec:sed} for references). For J0131-0321, J1034+2033, PSO352-15, J2228+0110 we show (broken) power-law radio emissions, with slopes ($\alpha$) derived from fits in the literature. For PSO055-00, PSO135+16, we assume a median high$-z$ slope value (see Section \ref{sec:sed}). The _ solid light blue lines_ are the measured power-law functions, while we report with _ dashed lines_ the extrapolation to lower/higher frequencies. The _ light cyan_ lines highlight the regions encompassed by the 1$\sigma$ uncertainties on the power-law slopes. We show with _ light blue empty circles_ the flux extrapolated from the synchtrotron power-law functions at the ALMA frequency targeted in this work. (*fig:sed*)

</div>
<div id="div_fig2">

<img src="tmp_2411.11952/./fig_quant_distr_202408_cii-1.png" alt="Fig2" width="100%"/>

**Figure 2. -**  Distribution of $\cii$  emission line and properties for RL (_pink_), and RQ (_grey_) quasars.  (*fig:distrprop*)

</div>
<div id="div_fig3">

<img src="tmp_2411.11952/./fig_lfir_lciifir_20240826_v4.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\cii$-to-FIR luminosity ratio as a function of FIR luminosity. Observations of star-forming galaxies and LIRGs at $z<1$ are shown with _light blue crosses_, while we report star-forming galaxies at $1<z<5$ with _orange arrows_. RQ quasars from the literature are depicted with _grey points_. All references for these data are reported in Section \ref{sec:analysis_sample} and \ref{sec:analysis_lum}. We show in _circles_ RL quasars observations from \cite{khusanova2022}, \cite{venemans2020} and \cite{banados2024b} and in _squares_ the values derived in this work, considering that the total measured ALMA continuum flux is due only to dust, all color-coded for radio loudness parameter. We note that RL quasars occupy a similar parameter space of that of RQ quasars' hosts, with the exception of the sources J0131-0321 and J0410-0139 (with $L_{\rm $\cii$} / L_{\rm FIR} \sim 10^{-3.5-4}$). When considering FIR luminosities accounting for the synchrotron contribution-derived flux (_ red squares_ for sources from this work, _ red circle_ from \citealt{banados2024b}), even the above mentioned outliers move closer to the RQ quasars' hosts location.  (*fig:lfir_lciifir*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.11952"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\sorthelp}[1]$
$\newcommand{\juan}[1]{{\color{magenta} #1}}$
$\newcommand{\sergio}[1]{{\color{green} #1}}$
$\newcommand{\ralf}[1]{{\bf \color{magenta} RSK: #1}}$
$\newcommand{\planck}{\Planck}$
$\newcommand{\nh}{N_{\textsc{H}}}$
$\newcommand{\IRAS}{\textit{IRAS\/}}$
$\newcommand{\WMAP}{\textit{WMAP\/}}$
$\newcommand{\COBE}{\textit{COBE\/}}$
$\newcommand{\Spitzer}{\textit{Spitzer\/}}$
$\newcommand{\healpix}{{\sc HEALPix}}$
$\newcommand{\sextractor}{{\sc SExtractor}}$
$\newcommand{\hii}{\ion{H}{II}}$
$\newcommand{\viewangle}{\alpha}$
$\newcommand{\kps}{km s^{-1}}$
$\newcommand{\vLOS}{v_{\rm LOS}}$
$\newcommand{\planckurl}{\burl{http://www.rssd.esa.int/index.php?project=PLANCK&page=Planck_Collaboration}}$
$\newcommand{\commentproof}[1]$
$\newcommand{\bfc}\newcommand$
$\newcommand{\bfm}{\bf \color{magenta}}$
$\newcommand{\bfm}\newcommand$
$\newcommand{\Herschel}{\textit{Herschel}}$
$\newcommand{\deg}{\ifmmode^\circ\else^\circ\fi}$
$\newcommand{\pdeg}{\ifmmode \setbox0=\hbox{^{\circ}}\rlap{\hskip.11\wd0 .}^{\circ} \else \setbox0=\hbox{^{\circ}}\rlap{\hskip.11\wd0 .}^{\circ}\fi}$
$\newcommand{\arcs}{\ifmmode{^{\scriptstyle\prime\prime}} \else ^{\scriptstyle\prime\prime}\fi}$
$\newcommand{\arcm}{\ifmmode{^{\scriptstyle\prime}} \else ^{\scriptstyle\prime}\fi}$
$\newcommand\parcs{\saa=.07em \sbb=.03em$
$     \ifmmode \hbox{\rlap{.}}^{\scriptstyle\prime\kern -\sbb\prime}\hbox{\kern -\sa}$
$     \else \rlap{.}^{\scriptstyle\prime\kern -\sbb\prime}\kern -\sa\fi}$
$\newcommand\parcm{\saa=.08em \sbb=.03em$
$     \ifmmode \hbox{\rlap{.}\kern\saa}^{\scriptstyle\prime}\hbox{\kern-\sbb}$
$     \else \rlap{.}\kern\saa^{\scriptstyle\prime}\kern-\sbb\fi}$</div>



<div id="title">

# Kinetic tomography of the Galactic plane within 1.25 kiloparsecs from the Sun: The interstellar flows revealed by H${\sc i}$ and CO line emission and 3D dust

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.12257-b31b1b.svg)](https://arxiv.org/abs/2411.12257)<mark>Appeared on: 2024-11-20</mark> -  _32 pages, 43 figures, submitted to Astronomy & Astrophysics (15NOV2024)_

</div>
<div id="authors">

J. D. Soler, et al. -- incl., <mark>H. Beuther</mark>

</div>
<div id="abstract">

**Abstract:** We present a reconstruction of the line-of-sight motions of the local interstellar medium (ISM) based on the combination of a state-of-the-art model of the three-dimensional dust density distribution within 1.25 kpc from the Sun and the H ${\sc i}$ and CO line emission within Galactic latitudes $|b|$ $\leq$ 5 $\deg$ .We use the histogram of oriented gradient (HOG) method, a computer vision technique for evaluating the morphological correlation between images, to match the plane-of-the-sky 3D dust distribution across distances with the atomic and molecular line emission.We identify a significant correlation between the 3D dust model and the line emission.We employ this correlation to assign line-of-sight velocities to the dust across density channels and produce a face-on map of the local ISM radial motions with respect to the local standard of rest (LSR).We find that most of the material in the 3D dust model follows the large-scale pattern of Galactic rotation; however, we also report local departures from the rotation pattern with standard deviations of 12.1 and 6.1 $\kps$ for the H ${\sc i}$ and CO line emission, respectively.The mean kinetic energy densities of these streaming motions are around 0.68 and 0.18 eV/cm $^{3}$ from either gas tracer.Assuming homogeneity and isotropy in the velocity field, these values are within a factor of a few of the total kinetic energy density.These kinetic energy values are roughly comparable to other energy densities. thus confirming the near-equipartition introduced by the feedback loops connecting the physical processes in the local ISM.Yet, we find energy and momentum overdensities of around a factor of ten concentrated in the Radcliffe Wave, the Split, and other local density structures, particularly around the Vela C and Ara star-forming regions.Although we do not find evidence of the Local Spiral Arm impact in these overdensities, their distribution suggests the influence of other large-scale effects that, in addition to supernova feedback, shape the energy distribution and dynamics in the Solar neighborhood.

</div>

<div id="div_fig1">

<img src="" alt="Fig28.1" width="50%"/><img src="" alt="Fig28.2" width="50%"/>

**Figure 28. -** Maximum morphological correlation between the 10$\deg$$\times$ 10$\deg$ distance channels and the H{\sc i}(_ left_) and CO (_ right_) line emission in the range $-25$$<$\vLOS$<$ 25 $\kps$, as quantified by the direction-sensitive projected Rayleigh statistic ($V_{\rm d}$; Eq. \ref{eq:Vd}).
 (*fig:polarPRS*)

</div>
<div id="div_fig2">

<img src="" alt="Fig32.1" width="50%"/><img src="" alt="Fig32.2" width="50%"/>

**Figure 32. -** Line-of-sight velocity derived from the H{\sc i}(_ left_) and CO (_ right_) line emission associated to the distance channels in the 3D dust reconstruction using the HOG method ($v^{\rm HOG}_{\rm LOS}$).
The colored circles correspond to the position and $\vLOS$ for the five high-mass star-forming regions with VLBI parallax observations within the studied volume.
 (*fig:polarvLSRfromHIandCO*)

</div>
<div id="div_fig3">

<img src="" alt="Fig13.1" width="50%"/><img src="" alt="Fig13.2" width="50%"/>

**Figure 13. -** Comparison of the results obtained with $V$ and $V_{\rm d}$ for the mean 3D dust and the line emission toward the tile covering the Galactic longitude range 30$\deg$$<$$l$$<$ 40$\deg$.
 (*fig:comparisonVandVd*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.12257"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\nikhil}[1]{\textcolor{blue}{#1}}$
$\newcommand{\john}[1]{\textcolor{red}{#1}}$
$\newcommand{\common}[1]{\textcolor{purple}{#1}}$
$\newcommand{\mhalo}{{\rm{M}_{\rm{halo}}}}$
$\newcommand{\mst}{{\rm{M}_{*}}}$
$\newcommand{\msun}{{\rm{M}_{\odot}}}$
$\newcommand{\nikhil}[1]{\textcolor{blue}{#1}}$
$\newcommand{\john}[1]{\textcolor{red}{#1}}$
$\newcommand{\common}[1]{\textcolor{purple}{#1}}$
$\newcommand{\mhalo}{{\rm{M}_{\rm{halo}}}}$
$\newcommand{\mst}{{\rm{M}_{*}}}$
$\newcommand{\msun}{{\rm{M}_{\odot}}}$</div>



<div id="title">

# Estimating Dark Matter Halo Masses in Simulated Galaxy Clusters with Graph Neural Networks

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.12629-b31b1b.svg)](https://arxiv.org/abs/2411.12629)<mark>Appeared on: 2024-11-20</mark> -  _9 pages, 4 figures, accepted at the NeurIPS ML4PS 2024 workshop_

</div>
<div id="authors">

N. Garuda, J. F. Wu, D. Nelson, <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** Galaxies grow and evolve in dark matter halos. Because dark matter is not visible, galaxies' halo masses ( $\mhalo$ ) must be inferred indirectly. We present a graph neural network (GNN) model for predicting $\mhalo$ from stellar mass ( $\mst$ ) in simulated galaxy clusters using data from the IllustrisTNG simulation suite. Unlike traditional machine learning models like random forests, our GNN captures the information-rich substructure of galaxy clusters by using spatial and kinematic relationships between galaxy neighbour. A GNN model trained on the TNG-Cluster dataset and independently tested on the TNG300 simulation achieves superior predictive performance compared to other baseline models we tested. Future work will extend this approach to different simulations and real observational datasets to further validate the GNN model’s ability to generalise.

</div>

<div id="div_fig1">

<img src="tmp_2411.12629/./figs/distance_from_center_RMSE.png" alt="Fig2" width="100%"/>

**Figure 2. -** Validation set RMSE as a function of distance from cluster center. Results shown for the GNN (blue) and RF with $\mst$ and $\Delta_G$(orange).\vspace{-1em} (*fig:rmse-dist-from-center*)

</div>
<div id="div_fig2">

<img src="tmp_2411.12629/./figs/cluster_graph.png" alt="Fig3" width="100%"/>

**Figure 3. -** Spatial distribution of halos within the TNG-Cluster simulation. The middle panel shows the full simulation, and the left and right panels highlight two example galaxy clusters. The boundaries of these clusters are marked as blue and red boxes in the middle panel. (*fig:halo-distr-tng-cluster*)

</div>
<div id="div_fig3">

<img src="tmp_2411.12629/./figs/distance_from_center_RMSE.png" alt="Fig5" width="100%"/>

**Figure 5. -** Validation set RMSE as a function of distance from cluster center. Results shown for the GNN (blue) and RF with $\mst$ and $\Delta_G$(orange).\vspace{-1em} (*fig:rmse-dist-from-center*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.12629"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\mum}{\text{\mum}}$
$\newcommand{\Jybeam}{\text{Jy beam^{-1}}}$</div>



<div id="title">

# Peculiar Dust Emission within the Orion Molecular Cloud

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.12693-b31b1b.svg)](https://arxiv.org/abs/2411.12693)<mark>Appeared on: 2024-11-20</mark> -  _Accepted to ApJ, 18 pages, 4 figures, 6 Tables_

</div>
<div id="authors">

P. Nozari, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** It is widely assumed that dust opacities in molecular clouds follow a  power-law profile with an index, $\beta$ . Recent studies of the Orion Molecular Cloud (OMC) 2/3 complex, however, show a flattening in the spectral energy distribution (SED) at $ \lambda > 2$ mm implying non-constant indices on scales $\gtrsim$ 0.08 pc. The origin of this flattening is not yet known but it may be due to the intrinsic properties of the dust grains or contamination from other sources of emission. We investigate the SED slopes in OMC 2/3 further using observations of six protostellar cores with NOEMA from 2.9 mm to 3.6 mm and ALMA-ACA in Band 4 (1.9 -- 2.1 mm) and Band 5 (1.6 -- 1.8 mm)    on core and envelope scales of $\sim 0.02 - 0.08$ pc. We confirm flattened opacity indices between 2.9 mm and 3.6 mm for the six cores with $\beta \approx -0.16 - 1.45$ , which are notably lower than the $\beta$ values of $> 1.3$ measured for these sources on $0.08$ pc scales from single-dish data. Four sources have consistent SED slopes between the ALMA data and the NOEMA data.  We propose that these sources may have a significant fraction of emission coming from large dust grains in embedded disks, which biases the emission more at longer wavelengths. Two sources, however, had inconsistent slopes between the ALMA and NOEMA data, indicating  different origins of emission.These results highlight how care is needed when combining multi-scale observations or extrapolating single-band observations to other wavelengths.

</div>

<div id="div_fig1">

<img src="tmp_2411.12693/./figures/finalResult_smooth_NOEMA_ALMA_.png" alt="Fig3" width="100%"/>

**Figure 3. -** SEDs for the six sources with best-fit power-law slopes for the ALMA data (dashed) and NOEMA data (dotted). The yellow areas show the uncertainty on the ALMA slope from 10\% flux calibration errors. Solid lines show the modified blackbody functions from single-dish data (*Herschel* at 0.16 -- 0.5 mm and IRAM+GISMO at 2 mm) from \citet{Sadavoy2016}. (*fig:omc_result*)

</div>
<div id="div_fig2">

<img src="tmp_2411.12693/./figures/NOEMA_MUSTANG_Smooth.png" alt="Fig4" width="100%"/>

**Figure 4. -** SEDs for the six sources with best-fit power-law slopes for the NOEMA data (dotted). The 3 mm GBT MUSTANG \citep{mason2020} are included for comparison but were not used in the fitting.  {To facilitate a robust comparison, the NOEMA data were smoothed to match the 10.8 $\arcsec$ resolution of the GBT data.} (*fig:NOEMA_MUSTANG_result*)

</div>
<div id="div_fig3">

<img src="tmp_2411.12693/./figures/source_gallery.png" alt="Fig1" width="100%"/>

**Figure 1. -** Cleaned NOEMA images of all six protostellar cores in OMC 2/3 for the LSB-I baseband at a central frequency of 86.78157 GHz.   {Contours represent a 5$\sigma$ level.} The synthesized beam is plotted in the bottom left corner.   {These maps represent the unsmoothed NOEMA observations.} (*fig:source_gallery*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.12693"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

251  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
